# Importe

In [1]:
# general
import pandas as pd

# waits
import random
from time import sleep

# driver
import undetected_chromedriver.v2 as uc
from selenium import webdriver
import chromedriver_autoinstaller

# selenium
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# other
# from ssl import Options

# multi
from concurrent import futures


# Driver

In [2]:
def get_driver(ip="", port="", flag_proxy=False):
    # PATH = "C:\Program Files (x86)\chromedriver.exe"
    chrome_options = uc.ChromeOptions()

    # Removes navigator.webdriver flag
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")

    # prevent bugs and skip popups
    chrome_options.add_argument('--no-first-run --no-service-autorun --password-store=basic')
    chrome_options.add_argument("--disable-extensions")

    # uc - user agent
    # bot detection can determine if os in user-agent differs from real os
    # chrome_options.add_argument("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.5 Safari/605.1.15")
    # chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36")

    # add multiple download flag
    dl_multi = {'profile.default_content_setting_values.automatic_downloads': 1}
    chrome_options.add_experimental_option("prefs", dl_multi)

    # add download location
    dl_path = {"download.default_directory" : "C:\\Users\\lange\\OneDrive\\Dokumente\\Studium\\06_Abschlussarbeiten\\Seminar\\02_Download"}
    chrome_options.add_experimental_option("prefs", dl_path)

    # add profile
    # uc.Chrome(user_data_dir='C:\\Users\\lange\\AppData\\Local\\Google\\Chrome\\User Data\\Default')
    
    # add proxy if True
    # -1 to use first element after use of driver to scrape proxies
    if flag_proxy == True:       
        PROXY = str(ip + ":" + port)
        print(PROXY)
        chrome_options.add_argument('--proxy-server=http://%s' % PROXY)

    # # be careful with headless browsing
    # chrome_options.add_argument("--headless")
    
    # # additional options to allow downloading while in headless mode
    # prefs = {
    # "download.default_directory": "C:\\Users\\lange\\OneDrive\\Dokumente\\Studium\\06_Abschlussarbeiten\\Seminar\\02_Download",
    # "download.directory_upgrade": True,
    # "download.prompt_for_download": False,
    # "plugins.always_open_pdf_externally": True,
    # "profile.default_content_settings.popups": "0",
    # "profile.content_settings.exceptions.automatic_downloads.*.setting": "1"
    # }
    # args = ["--no-sandbox",
    #         "--disable-dev-shm-usage"]
    
    # chrome_options.add_experimental_option("prefs", prefs)
    # chrome_options.add_experimental_option("args", args)
    # # chrome_options.add_experimental_option('useAutomationExtension', False)

    # webdriver vs. undetected chromedriver
    # driver = webdriver.Chrome(executable_path=PATH, options=chrome_options)
    driver = uc.Chrome(options=chrome_options)
    driver.maximize_window()
    
    return driver

# Proxies

In [3]:
# keine Verwendung, da public proxies zu 99% nicht funktionieren
# chromedriver_autoinstaller.install() # To update your chromedriver automatically
# driver = webdriver.Chrome("C:\\Program Files (x86)\\chromedriver")

# Get free proxies for rotating
def get_list_proxies(driver):
    driver.get('https://sslproxies.org')

    table = driver.find_element(By.TAG_NAME, 'table')
    thead = table.find_element(By.TAG_NAME, 'thead').find_elements(By.TAG_NAME, 'th')
    tbody = table.find_element(By.TAG_NAME, 'tbody').find_elements(By.TAG_NAME, 'tr')

    headers = []
    for th in thead:
        headers.append(th.text.strip())

    proxies = []
    for tr in tbody:
        proxy_data = {}
        tds = tr.find_elements(By.TAG_NAME, 'td')
        for i in range(len(headers)):
            proxy_data[headers[i]] = tds[i].text.strip()
        proxies.append(proxy_data)
    
    driver.quit()
    return proxies

In [4]:
def get_dlb_elements(driver):
    # try to extract clickable elements, if not possible, because their is no link, set empty
    try:
        AD_elements = WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located((By.LINK_TEXT, "AD")))
    except:
        AD_elements = []
    try:
        CD_elements = WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located((By.LINK_TEXT, "CD")))
    except:
        CD_elements = []
    try:
        HD_elements = WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located((By.LINK_TEXT, "HD")))
    except:
        HD_elements = []
    
    # create dict of elements from extracted lists
    dict_dlb_elements = {}
    dict_dlb_elements["AD"] = AD_elements
    dict_dlb_elements["CD"] = CD_elements
    dict_dlb_elements["HD"] = HD_elements
    
    return driver, dict_dlb_elements
    

In [5]:
def scrape_dlb_elements(driver, dict_dlb_elements, click_count):
    print(dict_dlb_elements)
    
    for key, elements in dict_dlb_elements.items():
        print(key)
        if not elements:
            continue
        index_elements = len(elements) - 1
        q = 0
        while q <= index_elements:
            try:
                # get and access fresh elements with old key, structure is the same
                driver, fresh_dict_dlb_elements = get_dlb_elements(driver)
                fresh_elements = fresh_dict_dlb_elements[key]
                fresh_elements[q].click()
                sleep(round(random.uniform(0.31, 2.11), 2))
                # download and go back
                driver.find_element(By.ID, "form:kostenpflichtigabrufen").click()
                click_count += 1
                driver.back()
                print("sucessfully downloaded {0} \nat q= {1}".format(elements[q], q))
            except Exception as exc:
                print('ERROR while downloading {0} \nat q= {1} \nexception: {2}'.format(elements[q], q, exc))
            q += 1
            print("q after download: ", q)    
    
    
            
    return  driver, click_count

# Scraper

In [7]:
def scrape(swort, nsitz, ip, port, click_count, select_ip, valid_proxy, flag_proxy=False):
    print(swort)
    print(nsitz)
    
    try:
        driver = get_driver(flag_proxy=flag_proxy, ip=ip, port=port)
        URL = "https://www.handelsregister.de/rp_web/erweitertesuche.xhtml"
        # URL = "https://bot.sannysoft.com/"    # test site for bot detection
        # URL = "https://nowsecure.nl"          # test site with max anti-bot protection
        driver.get(URL)
        
        # input
        input_swort = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.ID, "form:schlagwoerter")))
        input_nsitz = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.ID, "form:NiederlassungSitz")))
        input_swort.clear()
        input_nsitz.clear()

        input_swort.send_keys(swort)
        sleep(round(random.uniform(0.31, 2.11), 2))
        input_nsitz.send_keys(nsitz)
        sleep(round(random.uniform(0.31, 2.11), 2))

        # submit does not work, so click button
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(round(random.uniform(0.31, 2.11), 2))
        driver.find_element(By.ID, "form:btnSuche").click()
        sleep(round(random.uniform(0.31, 2.11), 2))

        # first get_dlb_elements
        driver, dict_dlb_elements = get_dlb_elements(driver)
        
        # DK_elements = WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located((By.LINK_TEXT, "DK")))
        # AD_elements = WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located((By.LINK_TEXT, "UT")))
        # AD_elements = WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located((By.LINK_TEXT, "VÖ")))
        # AD_elements = WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located((By.LINK_TEXT, "SI")))
        
    except Exception as exc:
        select_ip +=1
        valid_proxy = False
        print('ERROR with ip: {0} and port: {1} generated an exception: {2}\n try next ...'.format(ip, port, exc))
        driver.quit()
        return click_count, select_ip, valid_proxy
    
    driver, click_count = scrape_dlb_elements(driver, dict_dlb_elements, click_count)
    print("reached quit")
    driver.quit()
    valid_proxy = True   
     
    return driver, click_count, select_ip, valid_proxy

# Data

In [9]:
# example premium proxies
list_proxies = [#{'IP Address': '45.94.47.66', 'Port': '8110'},
                {'IP Address': '45.155.68.129', 'Port': '8133'},
                {'IP Address': '185.199.228.220', 'Port': '7300'}]

In [11]:
# Einlesen der gelabelten URLs, samt Registerinformationen zur Suche im Handelsregister
df_terms = pd.read_excel("C:\\Users\\lange\\OneDrive\\Dokumente\\Studium\\06_Abschlussarbeiten\\Seminar\\UN_HRB.xlsx")
df_terms.sample(5)

,name,regricht,regart,regnum,url,label
13,Henkel AG & Co. KGaA,Düsseldorf,HRB,4724,https://www.henkel.de/,Consumer Goods
7,Bau-Service Baur GmbH,Ulm,HRB,490392,https://bauservice-baur.de/,Construction
99,Schramm Bauschutt-Recycling GmbH & Co.,Coburg,HRB,2320,https://www.bauschuttdeponie-schramm.de/,Renewables & Environment
108,​PHOENIX Pharma SE,Mannheim,HRB,727494,https://www.phoenixgroup.eu/de/,Wholesale
17,Nestlé Deutschland AG,Frankfurt am Main,HRB,28163,https://www.nestle.de/,Consumer Goods


In [14]:
# example df
data = [['INTENSE AG', 'Würzburg'], ['snapADDY GmbH', 'Würzburg']]
df_terms = pd.DataFrame(data, columns=['swort', 'nsitz'])
df_terms

,swort,nsitz
0,INTENSE AG,Würzburg
1,snapADDY GmbH,Würzburg


# Calls

Läuft mit funktionierenden Proxies allerdings sind die Proxies aus dem Beispiel inzwischen wieder veraltet.

In [15]:
# call scraper
# check different methods for faster applying functions to data
# https://www.ml4devs.com/articles/pandas-dataframe-apply-function-iterate-over-rows/

select_ip = 0
click_count = 0
for index, row in df_terms.iterrows():
    
    valid_proxy = False
    while valid_proxy == False:
        
        if click_count >= 55:
            select_ip +=1
            click_count = 0
    
        dict_proxy = list_proxies[select_ip]
        ip = dict_proxy["IP Address"]
        port = dict_proxy["Port"]
    
        try:
            driver, click_count, select_ip, valid_proxy = scrape(swort=row['swort'], 
                                                                nsitz=row['nsitz'], 
                                                                flag_proxy=True, 
                                                                ip=ip, 
                                                                port=port, 
                                                                click_count=click_count, 
                                                                select_ip=select_ip, 
                                                                valid_proxy=valid_proxy)
        except Exception as exc:
            print("ERROR {}".format(exc))

INTENSE AG
Würzburg
45.155.68.129:8133
ERROR with ip: 45.155.68.129 and port: 8133 generated an exception: Message: 
Stacktrace:
Backtrace:
	(No symbol) [0x0072F243]
	(No symbol) [0x006B7FD1]
	(No symbol) [0x005AD04D]
	(No symbol) [0x005DC0B0]
	(No symbol) [0x005DC22B]
	(No symbol) [0x0060E612]
	(No symbol) [0x005F85D4]
	(No symbol) [0x0060C9EB]
	(No symbol) [0x005F8386]
	(No symbol) [0x005D163C]
	(No symbol) [0x005D269D]
	GetHandleVerifier [0x009C9A22+2655074]
	GetHandleVerifier [0x009BCA24+2601828]
	GetHandleVerifier [0x007D8C0A+619850]
	GetHandleVerifier [0x007D7830+614768]
	(No symbol) [0x006C05FC]
	(No symbol) [0x006C5968]
	(No symbol) [0x006C5A55]
	(No symbol) [0x006D051B]
	BaseThreadInitThunk [0x75AD6BD9+25]
	RtlGetFullPathName_UEx [0x77108FD2+1218]
	RtlGetFullPathName_UEx [0x77108F9D+1165]

 try next ...
ERROR not enough values to unpack (expected 4, got 3)
INTENSE AG
Würzburg
45.155.68.129:8133
ERROR with ip: 45.155.68.129 and port: 8133 generated an exception: Message: no suc

# Other

In [ ]:
# MULTI Threading
id, swort, nsitz = get_terms(terms=terms)
# default number of threads is optimized for cpu cores 
# possible to set with `max_workers` like `futures.ThreadPoolExecutor(max_workers=...)`
with futures.ThreadPoolExecutor() as executor:     
    # store the id for each thread as a dict, so we can know which thread fails
    future_results = { id : executor.submit(scrape(swort, nsitz), terms) for id in terms }
    for id, future in future_results.items(): 
        try:        
           future.result() # can use `timeout` to wait max seconds for each thread  
        except Exception as exc: # can give a exception in some thread
           print('id {0} generated an exception: {1}'.format(id, exc))

In [ ]:
# PROXY TEST
PROXY = "174.138.16.96:8888" # IP:PORT or HOST:PORT

chrome_options = uc.ChromeOptions()
chrome_options.add_argument('--proxy-server=http://%s' % PROXY)

driver = uc.Chrome(options=chrome_options)
driver.get("http://whatismyipaddress.com")